In [25]:
import collections

In [26]:
Position = collections.namedtuple("Position", ("x", "y"))

In [27]:
numeric_keypad = dict({
  "A": Position(2, 3),
  "0": Position(1, 3),
  "1": Position(0, 2),
  "2": Position(1, 2),
  "3": Position(2, 2),
  "4": Position(0, 1),
  "5": Position(1, 1),
  "6": Position(2, 1),
  "7": Position(0, 0),
  "8": Position(1, 0),
  "9": Position(2, 0),
  "gap": Position(0, 3)
})

In [28]:
directional_keypad = dict({
  "A": Position(2, 0),
  "^": Position(1, 0),
  "v": Position(1, 1),
  "<": Position(0, 1),
  ">": Position(2, 1),
  "gap": Position(0, 0)
})

In [42]:
def get_keypresses(code, keypad):
  presses = ""
  cur = keypad["A"]

  for key in code:
    pos = keypad[key]
    up_right = False
    difference = Position(pos.x - cur.x, pos.y - cur.y)
    down_right = difference.y > 0 and difference.x > 0
    
    if (keypad["gap"] == Position(pos.x, cur.y) or   # can't go left first because gap
       (down_right and keypad["gap"] != Position(cur.x, pos.y))):
      presses += "^" * (-1 * difference.y)
      presses += "v" * difference.y
      presses += "<" * (-1 * difference.x)
      presses += ">" * difference.x
    else:
      presses += "<" * (-1 * difference.x)
      presses += ">" * difference.x
      presses += "^" * (-1 * difference.y)
      presses += "v" * difference.y
    presses += "A"
    cur = pos

  return presses

In [30]:
def get_possible_keypresses(code, keypad):
  presses = [""]
  cur = keypad["A"]

  for key in code:
    pos = keypad[key]
    difference = Position(pos.x - cur.x, pos.y - cur.y)
    dupe_presses = []
    # print(presses)
    for i in range(len(presses)):
      # print(presses[i])
      if (keypad["gap"] == Position(pos.x, cur.y)):
        presses[i] += "^" * (-1 * difference.y)
        presses[i] += "v" * difference.y
        presses[i] += "<" * (-1 * difference.x)
        presses[i] += ">" * difference.x
      else:
        if (difference.x != 0 and difference.y != 0 and 
            (keypad["gap"] != Position(cur.x, pos.y))):
          dupe_press = presses[i]
          dupe_press += "^" * (-1 * difference.y)
          dupe_press += "v" * difference.y
          dupe_press += "<" * (-1 * difference.x)
          dupe_press += ">" * difference.x
          dupe_press += "A"
          dupe_presses.append(dupe_press)
        
        presses[i] += "<" * (-1 * difference.x)
        presses[i] += ">" * difference.x
        presses[i] += "^" * (-1 * difference.y)
        presses[i] += "v" * difference.y
      presses[i] += "A"
    presses += dupe_presses
    cur = pos

  return presses

In [31]:
def simulate_presses(code, keypad):
  reverse_keypresses = {v: k for k, v in keypad.items()}

  presses = ""
  cur = keypad["A"]

  for move in code:
    if move == "^":
      cur = Position(cur.x, cur.y-1)
    elif move == "v":
      cur = Position(cur.x, cur.y+1)
    elif move == "<":
      cur = Position(cur.x-1, cur.y)
    elif move == ">":
      cur = Position(cur.x+1, cur.y)
    elif move == "A":
      presses += reverse_keypresses[cur]
    
  return presses

In [32]:
def get_numeric_code(code):
  return int(code[:-1])

In [33]:
def get_sub_codes(code):
  str_pointer = 0
  sub_codes = []

  while str_pointer < len(code):
    next_occr = code.find("A", str_pointer)
    sub_codes.append(code[str_pointer:next_occr + 1])
    str_pointer = next_occr + 1

  return sub_codes

In [34]:
def shortest_sequence(code, keypad, depth, seen):
  if depth == 0:
    return len(code)
  if (code, depth) in seen:
    return seen[(code, depth)]
  
  length = 0
  for sub_code in get_sub_codes(code):
    sub_code_lengths = []
    poss_seqs = get_possible_keypresses(sub_code, keypad)
    for poss_seq in poss_seqs:
      sub_code_lengths.append(shortest_sequence(poss_seq, keypad, depth - 1, seen))

    length += min(sub_code_lengths)

  seen.update({(code, depth): length})
  return length
  

In [35]:
# Read input
codes = []

with open("input.txt", "r") as file:
  line = file.readline()
  while line:
    codes.append(line.strip())
    line = file.readline()

In [36]:
# Part 1

complexity = 0

for code in codes:
  press_seq = []
  press_seq.append(get_keypresses(code, numeric_keypad))
  for i in range(2):
    press_seq.append(get_keypresses(press_seq[i], directional_keypad))

  length = len(press_seq[-1])
  complexity += length * get_numeric_code(code)
print(complexity)

176870


In [37]:
# Part 2
total = 0
cache = dict()

for code in codes:
  poss_num_codes = get_possible_keypresses(code, numeric_keypad)

  lengths = []
  for poss_num_code in poss_num_codes:
    lengths.append(shortest_sequence(poss_num_code, directional_keypad, 25, cache))

  total += min(lengths) * get_numeric_code(code)

total

223902935165512

In [38]:
####################################################################################

In [39]:
complexity = 0

# codes = ["035A", "262A", "191A", "373A"]
TOTAL_D_PADS = 5

for code in codes:
  press_seq = []
  press_seq.append(get_keypresses(code, numeric_keypad, TOTAL_D_PADS))
  for i in range(TOTAL_D_PADS):
    press_seq.append(get_keypresses(press_seq[i], directional_keypad, TOTAL_D_PADS-i))

  length = len(press_seq[-1])
  complexity += length * get_numeric_code(code)
  print(length, code)

print(complexity)

TypeError: get_keypresses() takes 2 positional arguments but 3 were given

In [43]:
# 3744542434

# Part 2
TOTAL_D_PADS = 25
CACHE_D_PADS = int(TOTAL_D_PADS / 2)

complexity = 0
seen = dict()

# codes = ["035A", "262A", "191A", "373A"]

for code in codes:
  press_seq = []
  press_seq.append(get_keypresses(code, numeric_keypad, TOTAL_D_PADS))
  for _ in range(CACHE_D_PADS):
    press_seq.append(get_keypresses(press_seq[-1], directional_keypad, TOTAL_D_PADS - i))

  length = 0
  str_pointer = 0
  while str_pointer < len(press_seq[-1]):
    next_occr = press_seq[-1].find("A", str_pointer)
    sub_code = press_seq[-1][str_pointer:next_occr + 1]

    if not sub_code in seen:
      press_sub_seq = [sub_code]
      for i in range(TOTAL_D_PADS - CACHE_D_PADS):
        press_sub_seq.append(get_keypresses(press_sub_seq[-1], directional_keypad, TOTAL_D_PADS - CACHE_D_PADS - i))
      seen.update({sub_code: len(press_sub_seq[-1])})

    length += seen[sub_code]
    str_pointer = next_occr + 1
    
  complexity += length * get_numeric_code(code)
  print(length, code)

print(complexity)

TypeError: get_keypresses() takes 2 positional arguments but 3 were given

In [17]:
# 3744542434

# 192594543284430
# 196181565506214

# 145372133534 540A
# 139142164708 582A
# 146910148574 169A
# 154351814006 593A
# 145977981840 579A
# 360361384268340
# Too High

# 142038754064 540A
# 135951637642 582A
# 140910447972 169A
# 148181579628 593A
# 142630761684 579A
# 350093533743912
# Too High

# 104398108852 540A
# 98217357614 582A
# 104819221230 169A
# 108866563640 593A
# 103839800326 579A
# 255933045926572
# Too High

# 103602492004 540A
# 97469427158 582A
# 102195244044 169A
# 106211168678 593A
# 103048344934 579A
# 252591763274392
# Unknown

# 78690823042 540A
# 74625360762 582A
# 78163929086 169A
# 81005491640 593A
# 78451776028 579A
# 192594543284430
# Unknown, but probably too high

In [18]:
# seen = dict()

# complexity = 0
# poss_press_seq = []

# for code in codes:
#   poss_press_seq = [get_possible_keypresses(code, numeric_keypad)]
#   for i in range(4):
#     new_press_seqs = []
#     for press_seq in poss_press_seq[-1]:
#       new_press_seqs += get_possible_keypresses(press_seq, directional_keypad)
#     poss_press_seq.append(new_press_seqs)

#   shortest_code = min(poss_press_seq[-1], key=len)

#   length = len(shortest_code)
#   complexity += length * get_numeric_code(code)
#   print(length, code)

# print(complexity)

In [19]:
seen = dict()

complexity = 0
poss_press_seq = []

for code in codes:
  poss_press_seq = [get_possible_keypresses(code, numeric_keypad)]
  for i in range(2):
    new_press_seqs = []
    for press_seq in poss_press_seq[-1]:
      new_press_seqs += get_possible_keypresses(press_seq, directional_keypad)
    poss_press_seq.append(new_press_seqs)

  shortest_code = min(poss_press_seq[-1], key=len)

  length = len(shortest_code)
  complexity += length * get_numeric_code(code)
  print(length, code)

print(complexity)

KeyboardInterrupt: 

In [577]:
# seen = dict()

# complexity = 0
# poss_press_seq = []

# codes = ["35A", "26A", "191A", "373A"]

# for code in codes:
#   poss_press_seq = [get_possible_keypresses(code, numeric_keypad)]
#   for i in range(3):
#     new_press_seqs = []
#     for press_seq in poss_press_seq[-1]:
#       new_press_seqs += get_possible_keypresses(press_seq, directional_keypad)
#     poss_press_seq.append(new_press_seqs)

#   new_codes = []
#   for press_seq in poss_press_seq[-1]:
#     new_codes.append(get_keypresses(press_seq, directional_keypad))

#   shortest_code = min(new_codes, key=len)

#   length = len(shortest_code)
#   complexity += length * get_numeric_code(code)
#   # print(shortest_code)
#   print(simulate_presses(shortest_code, directional_keypad))
#   print(simulate_presses(simulate_presses(shortest_code, directional_keypad), directional_keypad))
#   # print(simulate_presses(simulate_presses(simulate_presses(shortest_code, directional_keypad), directional_keypad), directional_keypad))
#   # print(simulate_presses(simulate_presses(simulate_presses(simulate_presses(shortest_code, directional_keypad), directional_keypad), directional_keypad), directional_keypad))
#   print(length, code)

# print(complexity)

In [578]:
# 184 540A
# 176 582A
# 186 169A
# 192 593A
# 186 579A
# 454776

In [579]:

# seen = dict()

# complexity = 0

# for code in codes:
#   poss_press_seq = [get_possible_keypresses(code, numeric_keypad)]
#   for i in range(2):
#     new_press_seqs = []
#     for press_seq in poss_press_seq[-1]:
#       new_press_seqs += get_possible_keypresses(press_seq, directional_keypad)
#       # new_press_seqs.append(get_keypresses(press_seq, directional_keypad))
#     poss_press_seq.append(new_press_seqs)

#   real_lengths = []
#   real_codes = []
#   for press_seq in poss_press_seq[-1]:
#     # length = 0
#     press_code = ""
#     str_pointer = 0

#     while str_pointer < len(press_seq):
#       next_occr = press_seq.find("A", str_pointer)
#       sub_code = press_seq[str_pointer:next_occr + 1]

#       if not sub_code in seen:
#         poss_press_sub_seq = [get_possible_keypresses(sub_code, directional_keypad)]
#         for i in range(2):
#           new_press_sub_seqs = []
#           for press_sub_seq in poss_press_sub_seq[-1]:
#             new_press_sub_seqs += get_possible_keypresses(press_sub_seq, directional_keypad)
#             # new_press_sub_seqs.append(get_keypresses(press_sub_seq, directional_keypad))
#           poss_press_sub_seq.append(new_press_sub_seqs)
        
#         # seen.update({sub_code: min(len(s) for s in poss_press_sub_seq[-1])})
#         seen.update({sub_code: min(poss_press_sub_seq[-1], key=len)})

#       # length += len(seen[sub_code])
#       press_code += seen[sub_code]
#       str_pointer = next_occr + 1
    
#     # real_lengths.append(length)
#     real_codes.append(press_code)

#   # length = min(real_lengths)
#   shortest_code = min(real_codes, key=len)
#   complexity += len(shortest_code) * get_numeric_code(code)
#   print(shortest_code)
#   print(simulate_presses(shortest_code, directional_keypad))
#   print(simulate_presses(simulate_presses(shortest_code, directional_keypad), directional_keypad))
#   print(simulate_presses(simulate_presses(simulate_presses(shortest_code, directional_keypad), directional_keypad), directional_keypad))
#   print(simulate_presses(simulate_presses(simulate_presses(simulate_presses(shortest_code, directional_keypad), directional_keypad), directional_keypad), directional_keypad))
#   print(simulate_presses(simulate_presses(simulate_presses(simulate_presses(simulate_presses(shortest_code, directional_keypad), directional_keypad), directional_keypad), directional_keypad), directional_keypad))
#   print(len(shortest_code), code)

# print(complexity)

In [580]:

# seen = dict()

# complexity = 0

# for code in codes:
#   poss_press_seq = [get_possible_keypresses(code, numeric_keypad)]
#   for i in range(2):
#     new_press_seqs = []
#     for press_seq in poss_press_seq[-1]:
#       # new_press_seqs += get_possible_keypresses(press_seq, directional_keypad)
#       new_press_seqs.append(get_keypresses(press_seq, directional_keypad))
#     poss_press_seq.append(new_press_seqs)

#   real_lengths = []
#   for press_seq in poss_press_seq[-1]:
#     length = 0
#     str_pointer = 0

#     while str_pointer < len(press_seq):
#       next_occr = press_seq.find("A", str_pointer)
#       sub_code = press_seq[str_pointer:next_occr + 1]

#       if not sub_code in seen:
#         poss_press_sub_seq = [get_possible_keypresses(sub_code, directional_keypad)]
#         for i in range(2):
#           new_press_sub_seqs = []
#           for press_sub_seq in poss_press_sub_seq[-1]:
#             # new_press_sub_seqs += get_possible_keypresses(press_sub_seq, directional_keypad)
#             new_press_sub_seqs.append(get_keypresses(press_sub_seq, directional_keypad))
#           poss_press_sub_seq.append(new_press_sub_seqs)
        
#         seen.update({sub_code: min(len(s) for s in poss_press_sub_seq[-1])})

#       length += seen[sub_code]
#       str_pointer = next_occr + 1

#     real_lengths.append(length)

#   length = min(real_lengths)
#   complexity += length * get_numeric_code(code)
#   print(length, code)

# print(complexity)

In [40]:
len(cache)

649

In [41]:
cache

{('<vA<AA>>^A', 1): 26,
 ('v<A<AA>>^A', 1): 26,
 ('vA^A', 1): 10,
 ('vA<^A>A', 1): 19,
 ('vA^<A>A', 1): 19,
 ('v<<A>A>^A', 2): 55,
 ('<A>vA^A', 1): 19,
 ('<Av>A^A', 1): 19,
 ('v<<A>A^>A', 2): 55,
 ('vAA<^A>A', 1): 20,
 ('vAA^<A>A', 1): 20,
 ('v<<A>>^A', 2): 46,
 ('A', 1): 1,
 ('A', 2): 1,
 ('<vA>^A', 1): 16,
 ('v<A>^A', 1): 16,
 ('<vA^>A', 1): 16,
 ('v<A^>A', 1): 16,
 ('v<<A>^A>A', 1): 21,
 ('vAA<^A>A', 2): 48,
 ('<Av<A>>^A', 1): 25,
 ('vAA^<A>A', 2): 52,
 ('<vA<AA>>^A', 3): 150,
 ('v<<A>A>^A', 1): 21,
 ('v<<A>A^>A', 1): 21,
 ('v<<A>>^A', 1): 18,
 ('<vA<A>>^A', 2): 59,
 ('<vA<A>>^A', 1): 25,
 ('v<A<A>>^A', 1): 25,
 ('v<A<A>>^A', 2): 63,
 ('v<A<AA>>^A', 3): 154,
 ('<vA>^A', 2): 40,
 ('v<A>^A', 2): 44,
 ('<vA^>A', 2): 40,
 ('v<A^>A', 2): 44,
 ('<A>A', 2): 28,
 ('vA^A', 3): 68,
 ('v<<A>^A>A', 2): 55,
 ('<A>A', 1): 12,
 ('vA^A', 2): 28,
 ('vA<^A>A', 3): 123,
 ('<Av<A>>^A', 2): 63,
 ('vA^<A>A', 3): 131,
 ('v<<A>A>^A', 4): 341,
 ('vA<A>^A', 2): 53,
 ('vA<A^>A', 2): 53,
 ('<A>vA^A', 3): 127,
